In [1]:
from pyopenms import *
import pyopenms as pms
import pandas as pd
import numpy as np
import os
import glob
import os

print(os.getcwd())
    
# here was a typo alinged vs aligner, strange extension xml instead of featureXML
input_feature_files = sorted(glob.glob('results/consensus/interim/MapAligner*featureXML*'))
print(input_feature_files)

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

use_centroid_rt= False
use_centroid_mz= True
protein_ids = []
peptide_ids= []

mapper = pms.IDMapper()

input_mzml_files= sorted(glob.glob("results/interim/*.mzML"))
print(input_mzml_files)
for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)

    for fmap in feature_maps:
        if os.path.basename(fmap.getMetaValue('spectra_data')[0].decode()) == os.path.basename(filename):
            peptide_ids = []
            protein_ids = []
            
            mapper.annotate(fmap, peptide_ids, protein_ids, use_centroid_rt, use_centroid_mz, exp)
            print(fmap.getMetaValue('spectra_data')[0].decode())
        featureidx_file = os.path.join("results", "", "consensus", "", "interim", "", 'IDMapper_' + os.path.basename(fmap.getMetaValue('spectra_data')[0].decode())[19:-5] +".featureXML")
        FeatureXMLFile().store(featureidx_file, fmap)

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


/Users/eeko/Desktop/pyOpenMS_untargeted_metabolomics
[]
['results/interim/precursorcorrected_Epemicins.mzML', 'results/interim/precursorcorrected_GermicidinA.mzML', 'results/interim/precursorcorrected_GermicidinB.mzML']
